<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ml801-project-NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

In [2]:
!apt install git-lfs

from huggingface_hub import notebook_login

notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

def calculate_cross_entropy(model, tokenizer, sentence, translation, device):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=tokenizer(translation, return_tensors="pt", padding=True, truncation=True).input_ids.to(device))
    loss = outputs.loss.item()
    return loss

def dual_ce_filtering(corpus, model, tokenizer, threshold, device):
    model.to(device)
    filtered_corpus = []

    for sentence_lang1, sentence_lang2 in corpus:
        ce_lang1 = calculate_cross_entropy(model, tokenizer, sentence_lang1, sentence_lang2, device)
        ce_lang2 = calculate_cross_entropy(model, tokenizer, sentence_lang2, sentence_lang1, device)

        if ce_lang1 < threshold and ce_lang2 < threshold:
            filtered_corpus.append((sentence_lang1, sentence_lang2))

    return filtered_corpus

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [5]:
from datasets import load_dataset

dataset = load_dataset("un_multi", "ar-en")

Generating train split:   0%|          | 0/9759125 [00:00<?, ? examples/s]

In [6]:
# Sample 100,000 rows from the training set
# sampled_dataset = dataset['train'].shuffle(seed=42).select(range(100000))

# Separate Arabic and English into two different columns
def separate_languages(example):
    return {
        'arabic': example['translation']['ar'],
        'english': example['translation']['en']
    }

parallel_dataset = dataset.map(separate_languages)

# Check the first few entries
print(parallel_dataset['arabic'][5])
print(parallel_dataset['english'][5])

Map:   0%|          | 0/9759125 [00:00<?, ? examples/s]

KeyError: ignored

In [8]:
parallel_dataset = parallel_dataset.map(remove_columns=['translation'])

Map:   0%|          | 0/9759125 [00:00<?, ? examples/s]

In [7]:
parallel_dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'arabic', 'english'],
        num_rows: 9759125
    })
})

In [9]:
parallel_dataset.push_to_hub("arb-eng-parallel")

Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1085 [00:00<?, ?ba/s]

In [ ]:
threshold = 0.5  # Set your threshold based on your specific needs
device = "cuda" if torch.cuda.is_available() else "cpu"

filtered_dataset = dual_ce_filtering(parallel_dataset['train'], model, tokenizer, threshold, device)
# print(filtered_dataset)

In [12]:
parallel_dataset

DatasetDict({
    train: Dataset({
        features: ['arabic', 'english'],
        num_rows: 9759125
    })
})